In [92]:
import requests
import pandas as pd
import numpy as np

In [83]:
z = [ requests.get('https://hacker-news.firebaseio.com/v0/item/'+str(i)+'.json').json() for i in range(19588895,19588885,-1)]

In [84]:
z[3]

{'by': 'rrauenza',
 'descendants': 0,
 'id': 19588892,
 'score': 1,
 'time': 1554519317,
 'title': 'How to tell when procrastinating is anxiety',
 'type': 'story',
 'url': 'https://qz.com/work/1584023/how-to-tell-when-procrastinating-is-actually-anxiety/'}

In [54]:
comments= pd.DataFrame(columns = ["by", "id","text","time"])
story = pd.DataFrame(columns = ["by","id","title","time","score","url"])

In [71]:
comments

,by,id,text,time


In [72]:
story

,by,id,title,time,score,url


In [198]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [137]:
r = requests.get('https://hacker-news.firebaseio.com/v0/item/19590311.json').json()

In [144]:
try:
    if 'deleted' in r.keys():
        pass
    else:
        "works"
except:
    print("pass")

pass


In [114]:
r2 = requests.get('https://hacker-news.firebaseio.com/v0/item/19598733.json').json()
np.nan if r2["deleted"] == True else False

KeyError: 'deleted'

In [75]:
maxitem = requests.get("https://hacker-news.firebaseio.com/v0/maxitem.json?print=pretty").json()

In [197]:
maxitem

19599263

In [146]:
def hn_scrape(i):
    r = requests.get('https://hacker-news.firebaseio.com/v0/item/'+str(i)+'.json').json()
    try:
        if ('deleted' in r.keys()):
            pass
        else:
            if r["type"] == 'comment':
                t = (r["by"],r["id"],r["text"],r["time"])
                comments_list.append(t)
            elif (r["type"] == "story") & ('url' in r.keys()):
                t = (r["by"],r["id"], r["title"],r["time"],r["score"],r["url"])
                story_list.append(t)
            elif r["type"] == "story" :
                t = (r["by"],r["id"], r["title"],r["time"],r["score"],np.nan)
                story_list.append(t)
    except:
        pass
                      

In [147]:
maxitem = requests.get("https://hacker-news.firebaseio.com/v0/maxitem.json?print=pretty").json()
comments_list = []
story_list =[]
item = range(maxitem,maxitem-100000,-1)
for i in log_progress(item):
    hn_scrape(i)

In [148]:
comments_list

[('fencepost',
  19599263,
  '&quot;Everything we could&quot; [and more than we should]<p>I can&#x27;t help but wonder if there was anyone with medical training acting as a patient advocate in this and similar situations.',
  1554665023),
 ('herohamp',
  19599262,
  'This. Or just some way to keep track of a conversation without constantly refreshing',
  1554665022),
 ('dukoid',
  19599261,
  'Dual citizenship is discouraged but I don&#x27;t think officials would care enough to actually &quot;force&quot; anybody... <a href="https:&#x2F;&#x2F;www.dw.com&#x2F;cda&#x2F;en&#x2F;dual-citizenship-granted-to-most-naturalized-germans&#x2F;a-45030118" rel="nofollow">https:&#x2F;&#x2F;www.dw.com&#x2F;cda&#x2F;en&#x2F;dual-citizenship-granted-to-most-n...</a>',
  1554665020),
 ('ogdrhr',
  19599260,
  'Yes, there&#x27;s no way to win as the hiring entity, you just open yourself up to discrimination lawsuits. So it&#x27;s better to do the inhuman thing and say nothing!',
  1554665010),
 ('changchu

In [149]:
comments= pd.DataFrame(columns = ["by", "id","text","time"], data=comments_list)
story = pd.DataFrame(columns = ["by","id","title","time","score","url"],data=story_list)

In [171]:
comments.shape

(86074, 4)

In [172]:
story.shape

(11192, 6)

In [174]:
comments["id"].tail()

86069    19499268
86070    19499267
86071    19499266
86072    19499265
86073    19499264
Name: id, dtype: int64

In [178]:
import pickle
filename = 'hn_story_19499274'
outfile = open(filename, 'wb')
pickle.dump(story,outfile)
outfile.close()

In [181]:
file_name = 'hn_story_19499274'
infile = open(filename,'rb')
test = pickle.load(infile)
infile.close()

In [254]:
min(comments.id.min(), story.id.min())

18790920

In [260]:
min_item = min(comments.id.min(), story.id.min())
comments_list = []
story_list =[]
item = range(min_item,min_item-150000,-1)
for i in log_progress(item):
    hn_scrape(i)

In [261]:
len(comments_list), len(story_list)

(126611, 19295)

In [262]:
comments = comments.append(pd.DataFrame(data= comments_list, columns=["by", "id","text","time"] ))
import pickle
filename = 'hn_comments_df'
outfile = open(filename, 'wb')
pickle.dump(comments,outfile)
outfile.close()

In [263]:
story = story.append(pd.DataFrame(data=story_list, columns = ["by","id","title","time","score","url"]))
filename = 'hn_story_df'
outfile = open(filename, 'wb')
pickle.dump(story,outfile)
outfile.close()

In [264]:
comments.shape, story.shape

((945740, 4), (149601, 6))